In [ ]:
help(webdriver.Chrome)

In [44]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd

In [59]:
# Setup WebDriver (example with Chrome)
driver = webdriver.Chrome()

driver.get("""https://baseballsavant.mlb.com/gamefeed?date=4/4/2024&gamePk=745844&chartType=pitch&legendType=pitchName&playerType=
           pitcher&inning=&count=&pitchHand=&batSide=&descFilter=&ptFilter=&resultFilter=&hf=liveAB&sportId=1#745844""")

# Wait up to 10 seconds for the dropdown to be available
dropdown = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "liveAB-matchups_745844"))
)

# Now that we've waited for it, we can proceed as before
select = Select(dropdown)

TEMP_DATA = []

num_options = len(select.options)

for index in range(num_options):
    # Re-find the dropdown and recreate the Select object each iteration
    dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "liveAB-matchups_745844")))
    select = Select(dropdown)
    
    select.select_by_index(index)
    option_text = select.options[index].text
    
    header = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, 'liveABTable_745844_th-0'))
)

    # Now, use the found header element to navigate to the ancestor table
    table = header.find_element(By.XPATH, './ancestor::table')

    headers = [th.text for th in table.find_element(By.TAG_NAME, 'thead').find_elements(By.TAG_NAME, 'th')]

    PITCH_DATA = []
    # Extract data from the table
    # Iterate over each row in the tbody (assuming there's only one tbody)
    for row in table.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr'):
        # For each row, extract each cell's text
        PITCHES = {headers[index]: cell.text for index, cell in enumerate(row.find_elements(By.TAG_NAME, 'td'))}
        PITCH_DATA.append(PITCHES)

    df_temp = pd.DataFrame(PITCH_DATA, columns=headers)
    df_temp['AT_BAT'] = option_text  # Add the dropdown option as a column
    TEMP_DATA.append(df_temp)

MATCHUPS = pd.concat(TEMP_DATA, ignore_index=True)
print(MATCHUPS)

driver.quit()


     #       Pitch Type           Result Pitch Vel (MPH)  Spin VBreak (In.)  \
0    1         Splitter    Called Strike            86.6  1952           29   
1    2         Splitter             Ball            85.2  1714           30   
2    3  4-Seam Fastball             Ball            93.1  2478           12   
3    4  4-Seam Fastball    Called Strike            93.1  2499           10   
4    5  4-Seam Fastball         Foul Tip            93.6  2453           10   
..  ..              ...              ...             ...   ...          ...   
377  2         Changeup  Swinging Strike            81.8  1315           38   
378  3         Changeup             Ball            82.1  1368           40   
379  4  4-Seam Fastball             Foul            93.3  2138           13   
380  5  4-Seam Fastball             Foul            93.5  2081           13   
381  6         Changeup  In play, out(s)            82.3  1509           42   

       HBreak (In.)                                

In [60]:
MATCHUPS.to_csv('2024_04_04_DET_NYM.csv', index=False)


In [62]:
CASEY_MIZE_DET_NYM_2024_04_04 = MATCHUPS[MATCHUPS['AT_BAT'].str.contains('Casey Mize')]
CASEY_MIZE_DET_NYM_2024_04_04 = CASEY_MIZE_DET_NYM_2024_04_04[['Pitch Type', '#', 'AT_BAT']]
CASEY_MIZE_DET_NYM_2024_04_04.to_csv('CASEY_MIZE_DET_NYM_2024_04_04.csv', index=False)